A simple example of starting a dask cluster with `mpirun` and having each worker in the cluster create an array, pass that array down to MPI, and scattering the arrays to each other.

In [1]:
from dask.distributed import Client, wait
import numpy as np

*Note:* It's important that your dask-mpi cluster has been run with `nthreads [>1]`. Otherwise the collective communications will deadlock.

In [2]:
client = Client(scheduler_file = "cluster.json", direct_to_workers = True)

In [3]:
client

Client Scheduler: tcp://10.2.171.62:8786 Dashboard: http://10.2.171.62:8787/status,Cluster Workers: 4 Cores: 40 Memory: 62.99 GB


In [4]:
def parse_host_port(address):
    if '://' in address:
        address = address.rsplit('://', 1)[1]
    host, port = address.split(':')
    port = int(port)
    return host, port

In [5]:
workers = [parse_host_port(worker) for worker in list(client.has_what().keys())]

In [6]:
from mpi4py import MPI

def get_ranks(l):
    try:
        comm = MPI.COMM_WORLD
        rank = comm.Get_rank()
        return rank
    
    except Exception as e:
        print(str(e))

In [7]:
def get_size():
    comm = MPI.COMM_WORLD
    return comm.Get_size()

In [8]:
# How many workers recognized in the MPI world?
client.submit(get_size).result()

4

### Get worker ranks

In [9]:
# Sending a unique value to each worker to make sure the function gets propagated to all the workers
pmpi = [(client.submit(get_ranks, ident, workers = [worker]), worker) for worker, ident in zip(workers, range(len(workers)))]
pmpi

[(<Future: status: pending, key: get_ranks-8ae7011239528d98d0bd607693d47dd8>,
  ('10.2.171.62', 42946)),
 (<Future: status: pending, key: get_ranks-188d7dd5a1ec5208090f7e8ee66f84e2>,
  ('10.2.171.62', 44392)),
 (<Future: status: pending, key: get_ranks-565b9238baefa743702c96630fc21847>,
  ('10.2.171.62', 37481)),
 (<Future: status: pending, key: get_ranks-70587cf10aeb1e8d699fb80c0e1420d9>,
  ('10.2.171.62', 41692))]

In [10]:
ranks = dict([(r.result(), worker) for r, worker in pmpi])

In [11]:
ranks

{0: ('10.2.171.62', 44392),
 1: ('10.2.171.62', 42946),
 2: ('10.2.171.62', 37481),
 3: ('10.2.171.62', 41692)}

First let's push some data to the workers for them to send to each other.

In [12]:
def create_np(rank):
    return np.arange(rank, rank+50)

In [13]:
np.arange(0, 50).dtype

dtype('int64')

In [14]:
res = [client.submit(create_np, rank, workers = [ranks[rank]]) for rank in ranks]
wait(res)

DoneAndNotDoneFutures(done={<Future: status: finished, type: ndarray, key: create_np-7027859d409dba0babb66601eced7865>, <Future: status: finished, type: ndarray, key: create_np-0c6df7c379f74635e67510f65812e6af>, <Future: status: finished, type: ndarray, key: create_np-6032cc0454f287929ff590d0f8f171b1>, <Future: status: finished, type: ndarray, key: create_np-af90b4d41e9be2943e7b705118bb7a32>}, not_done=set())

In [15]:
# Verify workers have generated data
client.who_has()

{'create_np-0c6df7c379f74635e67510f65812e6af': ('tcp://10.2.171.62:41692',),
 'create_np-6032cc0454f287929ff590d0f8f171b1': ('tcp://10.2.171.62:44392',),
 'create_np-7027859d409dba0babb66601eced7865': ('tcp://10.2.171.62:37481',),
 'create_np-af90b4d41e9be2943e7b705118bb7a32': ('tcp://10.2.171.62:42946',),
 'get_ranks-188d7dd5a1ec5208090f7e8ee66f84e2': ('tcp://10.2.171.62:44392',),
 'get_ranks-565b9238baefa743702c96630fc21847': ('tcp://10.2.171.62:37481',),
 'get_ranks-70587cf10aeb1e8d699fb80c0e1420d9': ('tcp://10.2.171.62:41692',),
 'get_ranks-8ae7011239528d98d0bd607693d47dd8': ('tcp://10.2.171.62:42946',)}

### Test point-to-point
An initial test to prove that we can do point-to-point communications across workers

In [16]:
list(range(1, 4))

[1, 2, 3]

In [17]:
def test_point_to_point(d):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    if rank == 0:
        data = d
        for i in range(1, 4):
            print("Sending to %s" % i)
            req = comm.isend(data, dest=i, tag=11)
        req.wait()
        return "sender"
    else:
        req = comm.irecv(source=0, tag=11)
        data = req.wait()
        return data

In [18]:
sender = [client.submit(test_point_to_point, future) for future in res]
sender

[<Future: status: pending, key: test_point_to_point-044b1d3d332ab20a232d39c4b7db5185>,
 <Future: status: pending, key: test_point_to_point-3815ad3c65ed8305c1e37c750418d756>,
 <Future: status: pending, key: test_point_to_point-3578a2bcecc0a0c5da560f9ffa771e23>,
 <Future: status: pending, key: test_point_to_point-dfd321bed292581121ee72d08962f308>]

In [19]:
client.who_has()

{'create_np-0c6df7c379f74635e67510f65812e6af': ('tcp://10.2.171.62:41692',),
 'create_np-6032cc0454f287929ff590d0f8f171b1': ('tcp://10.2.171.62:44392',),
 'create_np-7027859d409dba0babb66601eced7865': ('tcp://10.2.171.62:37481',),
 'create_np-af90b4d41e9be2943e7b705118bb7a32': ('tcp://10.2.171.62:42946',),
 'get_ranks-188d7dd5a1ec5208090f7e8ee66f84e2': ('tcp://10.2.171.62:44392',),
 'get_ranks-565b9238baefa743702c96630fc21847': ('tcp://10.2.171.62:37481',),
 'get_ranks-70587cf10aeb1e8d699fb80c0e1420d9': ('tcp://10.2.171.62:41692',),
 'get_ranks-8ae7011239528d98d0bd607693d47dd8': ('tcp://10.2.171.62:42946',),
 'test_point_to_point-044b1d3d332ab20a232d39c4b7db5185': ('tcp://10.2.171.62:44392',),
 'test_point_to_point-3578a2bcecc0a0c5da560f9ffa771e23': ('tcp://10.2.171.62:37481',),
 'test_point_to_point-3815ad3c65ed8305c1e37c750418d756': ('tcp://10.2.171.62:42946',),
 'test_point_to_point-dfd321bed292581121ee72d08962f308': ('tcp://10.2.171.62:41692',)}

In [20]:
client.gather(sender)

['sender',
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])]

### Test point-to-point with Numpy arrays 

In [21]:
def test_point_to_point_np(data):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    # automatic MPI datatype discovery
    if rank == 0:
        for i in range(1, 4):
            comm.Send(data, dest=i, tag=13)
        return data
    else:
        comm.Recv(data, source=0, tag=13)
        return data

In [22]:
nodes = [client.submit(test_point_to_point_np, future) for future in res]
nodes

[<Future: status: pending, key: test_point_to_point_np-9eb9c5f92e346067a2dd7c4b20c0f531>,
 <Future: status: pending, key: test_point_to_point_np-1ed611796daa89b8c29f223443a48f10>,
 <Future: status: pending, key: test_point_to_point_np-b66084dd29685bb77656272e1fc95112>,
 <Future: status: pending, key: test_point_to_point_np-3dba5d0677d20f086b1e836c32af051d>]

In [23]:
client.who_has()

{'create_np-0c6df7c379f74635e67510f65812e6af': ('tcp://10.2.171.62:41692',),
 'create_np-6032cc0454f287929ff590d0f8f171b1': ('tcp://10.2.171.62:44392',),
 'create_np-7027859d409dba0babb66601eced7865': ('tcp://10.2.171.62:37481',),
 'create_np-af90b4d41e9be2943e7b705118bb7a32': ('tcp://10.2.171.62:42946',),
 'get_ranks-188d7dd5a1ec5208090f7e8ee66f84e2': ('tcp://10.2.171.62:44392',),
 'get_ranks-565b9238baefa743702c96630fc21847': ('tcp://10.2.171.62:37481',),
 'get_ranks-70587cf10aeb1e8d699fb80c0e1420d9': ('tcp://10.2.171.62:41692',),
 'get_ranks-8ae7011239528d98d0bd607693d47dd8': ('tcp://10.2.171.62:42946',),
 'test_point_to_point-044b1d3d332ab20a232d39c4b7db5185': ('tcp://10.2.171.62:44392',),
 'test_point_to_point-3578a2bcecc0a0c5da560f9ffa771e23': ('tcp://10.2.171.62:37481',),
 'test_point_to_point-3815ad3c65ed8305c1e37c750418d756': ('tcp://10.2.171.62:42946',),
 'test_point_to_point-dfd321bed292581121ee72d08962f308': ('tcp://10.2.171.62:41692',),
 'test_point_to_point_np-1ed611796da

In [24]:
client.gather(nodes)

[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])]

In [25]:
client.who_has()

{'create_np-0c6df7c379f74635e67510f65812e6af': ('tcp://10.2.171.62:41692',),
 'create_np-6032cc0454f287929ff590d0f8f171b1': ('tcp://10.2.171.62:44392',),
 'create_np-7027859d409dba0babb66601eced7865': ('tcp://10.2.171.62:37481',),
 'create_np-af90b4d41e9be2943e7b705118bb7a32': ('tcp://10.2.171.62:42946',),
 'get_ranks-188d7dd5a1ec5208090f7e8ee66f84e2': ('tcp://10.2.171.62:44392',),
 'get_ranks-565b9238baefa743702c96630fc21847': ('tcp://10.2.171.62:37481',),
 'get_ranks-70587cf10aeb1e8d699fb80c0e1420d9': ('tcp://10.2.171.62:41692',),
 'get_ranks-8ae7011239528d98d0bd607693d47dd8': ('tcp://10.2.171.62:42946',),
 'test_point_to_point-044b1d3d332ab20a232d39c4b7db5185': ('tcp://10.2.171.62:44392',),
 'test_point_to_point-3578a2bcecc0a0c5da560f9ffa771e23': ('tcp://10.2.171.62:37481',),
 'test_point_to_point-3815ad3c65ed8305c1e37c750418d756': ('tcp://10.2.171.62:42946',),
 'test_point_to_point-dfd321bed292581121ee72d08962f308': ('tcp://10.2.171.62:41692',),
 'test_point_to_point_np-1ed611796da

### Test Bcast

In [26]:
def test_broadcast(d):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    
    try:
        if rank == 0:
            data = d
        else:
            data = None
        data = comm.bcast(data, root=0)
        return (rank, data)
    except Exception as e:
        print(str(e))

In [27]:
nodes = [client.submit(test_broadcast, future) for future in res]
nodes

[<Future: status: pending, key: test_broadcast-0e59256712a1e2a0fff111822b056d5f>,
 <Future: status: pending, key: test_broadcast-e34eca97ccaa1f13003521202a564c48>,
 <Future: status: pending, key: test_broadcast-cf6aa79dac2d0c60af48e70f4ecd7480>,
 <Future: status: pending, key: test_broadcast-d39defed807443cec9451befedd3a50d>]

In [28]:
client.who_has()

{'create_np-0c6df7c379f74635e67510f65812e6af': ('tcp://10.2.171.62:41692',),
 'create_np-6032cc0454f287929ff590d0f8f171b1': ('tcp://10.2.171.62:44392',),
 'create_np-7027859d409dba0babb66601eced7865': ('tcp://10.2.171.62:37481',),
 'create_np-af90b4d41e9be2943e7b705118bb7a32': ('tcp://10.2.171.62:42946',),
 'get_ranks-188d7dd5a1ec5208090f7e8ee66f84e2': ('tcp://10.2.171.62:44392',),
 'get_ranks-565b9238baefa743702c96630fc21847': ('tcp://10.2.171.62:37481',),
 'get_ranks-70587cf10aeb1e8d699fb80c0e1420d9': ('tcp://10.2.171.62:41692',),
 'get_ranks-8ae7011239528d98d0bd607693d47dd8': ('tcp://10.2.171.62:42946',),
 'test_broadcast-0e59256712a1e2a0fff111822b056d5f': ('tcp://10.2.171.62:44392',),
 'test_broadcast-cf6aa79dac2d0c60af48e70f4ecd7480': ('tcp://10.2.171.62:37481',),
 'test_broadcast-d39defed807443cec9451befedd3a50d': ('tcp://10.2.171.62:41692',),
 'test_broadcast-e34eca97ccaa1f13003521202a564c48': ('tcp://10.2.171.62:42946',),
 'test_point_to_point-044b1d3d332ab20a232d39c4b7db5185':

In [29]:
nodes

[<Future: status: finished, type: tuple, key: test_broadcast-0e59256712a1e2a0fff111822b056d5f>,
 <Future: status: finished, type: tuple, key: test_broadcast-e34eca97ccaa1f13003521202a564c48>,
 <Future: status: finished, type: tuple, key: test_broadcast-cf6aa79dac2d0c60af48e70f4ecd7480>,
 <Future: status: finished, type: tuple, key: test_broadcast-d39defed807443cec9451befedd3a50d>]

In [30]:
client.gather(nodes)

[(0, array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])),
 (1, array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])),
 (2, array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])),
 (3, array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]))]

### Test Scatter

In [31]:
def test_scatter(r):
    comm = MPI.COMM_WORLD
    size = comm.Get_size()
    rank = comm.Get_rank()

    sendbuf = None
    if rank == 0:
        sendbuf = np.empty([size, 100], dtype='i')
        sendbuf.T[:,:] = range(size)
    recvbuf = np.empty(100, dtype='i')
    comm.Scatter(sendbuf, recvbuf, root=0)
    return np.allclose(recvbuf, rank)

In [32]:
client.has_what(ranks[0][0] +":" + str(ranks[0][1]))

{'tcp://10.2.171.62:44392': ('test_broadcast-0e59256712a1e2a0fff111822b056d5f',
  'test_point_to_point-044b1d3d332ab20a232d39c4b7db5185',
  'create_np-6032cc0454f287929ff590d0f8f171b1',
  'test_point_to_point_np-9eb9c5f92e346067a2dd7c4b20c0f531',
  'get_ranks-188d7dd5a1ec5208090f7e8ee66f84e2')}

In [33]:
nodes = [client.submit(test_scatter, future) for future in res]
nodes

[<Future: status: pending, key: test_scatter-eac69e87ac23ddec1f4bffcd7ecf9c06>,
 <Future: status: pending, key: test_scatter-17459a8ed2dd202823e54bab2547b25f>,
 <Future: status: pending, key: test_scatter-e1d14fe85807cd436e2d69ec43f69214>,
 <Future: status: pending, key: test_scatter-4acfc261c395a7ebee8fe46b53782e80>]

In [34]:
client.who_has()

{'create_np-0c6df7c379f74635e67510f65812e6af': ('tcp://10.2.171.62:41692',),
 'create_np-6032cc0454f287929ff590d0f8f171b1': ('tcp://10.2.171.62:44392',),
 'create_np-7027859d409dba0babb66601eced7865': ('tcp://10.2.171.62:37481',),
 'create_np-af90b4d41e9be2943e7b705118bb7a32': ('tcp://10.2.171.62:42946',),
 'get_ranks-188d7dd5a1ec5208090f7e8ee66f84e2': ('tcp://10.2.171.62:44392',),
 'get_ranks-565b9238baefa743702c96630fc21847': ('tcp://10.2.171.62:37481',),
 'get_ranks-70587cf10aeb1e8d699fb80c0e1420d9': ('tcp://10.2.171.62:41692',),
 'get_ranks-8ae7011239528d98d0bd607693d47dd8': ('tcp://10.2.171.62:42946',),
 'test_broadcast-0e59256712a1e2a0fff111822b056d5f': ('tcp://10.2.171.62:44392',),
 'test_broadcast-cf6aa79dac2d0c60af48e70f4ecd7480': ('tcp://10.2.171.62:37481',),
 'test_broadcast-d39defed807443cec9451befedd3a50d': ('tcp://10.2.171.62:41692',),
 'test_broadcast-e34eca97ccaa1f13003521202a564c48': ('tcp://10.2.171.62:42946',),
 'test_point_to_point-044b1d3d332ab20a232d39c4b7db5185':

In [35]:
client.gather(nodes)

[True, True, True, True]

### Test Gather

In [36]:
def test_gather(r):
    comm = MPI.COMM_WORLD
    size = comm.Get_size()
    rank = comm.Get_rank()

    sendbuf = np.zeros(100, dtype='i') + rank
    recvbuf = None
    if rank == 0:
        recvbuf = np.empty([size, 100], dtype='i')
    comm.Gather(sendbuf, recvbuf, root=0)
    if rank == 0:
        return [np.allclose(recvbuf[i,:], i) for i in range(size)]

Notice the order of scheduling matters here. If the receiver is created after the senders, a deadlock will occur. 

In [37]:
nodes = [client.submit(test_gather, future) for future in res]

In [38]:
client.who_has()

{'create_np-0c6df7c379f74635e67510f65812e6af': ('tcp://10.2.171.62:41692',),
 'create_np-6032cc0454f287929ff590d0f8f171b1': ('tcp://10.2.171.62:44392',),
 'create_np-7027859d409dba0babb66601eced7865': ('tcp://10.2.171.62:37481',),
 'create_np-af90b4d41e9be2943e7b705118bb7a32': ('tcp://10.2.171.62:42946',),
 'get_ranks-188d7dd5a1ec5208090f7e8ee66f84e2': ('tcp://10.2.171.62:44392',),
 'get_ranks-565b9238baefa743702c96630fc21847': ('tcp://10.2.171.62:37481',),
 'get_ranks-70587cf10aeb1e8d699fb80c0e1420d9': ('tcp://10.2.171.62:41692',),
 'get_ranks-8ae7011239528d98d0bd607693d47dd8': ('tcp://10.2.171.62:42946',),
 'test_broadcast-0e59256712a1e2a0fff111822b056d5f': ('tcp://10.2.171.62:44392',),
 'test_broadcast-cf6aa79dac2d0c60af48e70f4ecd7480': ('tcp://10.2.171.62:37481',),
 'test_broadcast-d39defed807443cec9451befedd3a50d': ('tcp://10.2.171.62:41692',),
 'test_broadcast-e34eca97ccaa1f13003521202a564c48': ('tcp://10.2.171.62:42946',),
 'test_gather-3d36ec5a397e97bf637b85d5d57b2933': ('tcp:/

In [39]:
client.gather(nodes)

[[True, True, True, True], None, None, None]

### Test AllGather

In [43]:
def test_all_gather(d):
    comm = MPI.COMM_WORLD
    p = comm.Get_size()
    xg = np.zeros(p, dtype='i')
    comm.Allgather([d[0,0],  MPI.INT],
                   [xg, MPI.INT])
    return xg.sum()

In [44]:
nodes = [client.submit(test_all_gather, future) for future in res]
nodes

[<Future: status: pending, key: test_all_gather-ccc7521de3ee9da88389b87a62c7ad1a>,
 <Future: status: pending, key: test_all_gather-dd2e3e02d22728e54fb2a7678c1a7281>,
 <Future: status: pending, key: test_all_gather-5cfefe15933f53649369f19140eb619b>,
 <Future: status: pending, key: test_all_gather-1830945193103a5b620d1ec173895452>]

In [45]:
client.who_has()

{'create_np-0c6df7c379f74635e67510f65812e6af': ('tcp://10.2.171.62:41692',),
 'create_np-6032cc0454f287929ff590d0f8f171b1': ('tcp://10.2.171.62:44392',),
 'create_np-7027859d409dba0babb66601eced7865': ('tcp://10.2.171.62:37481',),
 'create_np-af90b4d41e9be2943e7b705118bb7a32': ('tcp://10.2.171.62:42946',),
 'get_ranks-188d7dd5a1ec5208090f7e8ee66f84e2': ('tcp://10.2.171.62:44392',),
 'get_ranks-565b9238baefa743702c96630fc21847': ('tcp://10.2.171.62:37481',),
 'get_ranks-70587cf10aeb1e8d699fb80c0e1420d9': ('tcp://10.2.171.62:41692',),
 'get_ranks-8ae7011239528d98d0bd607693d47dd8': ('tcp://10.2.171.62:42946',),
 'test_all_gather-1830945193103a5b620d1ec173895452': (),
 'test_all_gather-2caed08191b0fc49721bf8f440ca660d': (),
 'test_all_gather-5cfefe15933f53649369f19140eb619b': (),
 'test_all_gather-618885619a0a4624a9c1600e7a156bca': (),
 'test_all_gather-94bba4c9743199698e8e899e40d9e875': (),
 'test_all_gather-ac0d998a2ec15ee2da5efd4472808ebc': (),
 'test_all_gather-ccc7521de3ee9da88389b87a